In [81]:
import numpy as np
import requests
from bs4 import BeautifulSoup

从标签入手，先抓取所有的标签。写一个简单的函数。

In [46]:
def crawl_all_tags(url = None):
    all_problems = requests.get('https://leetcode.com/problemset/all/')
    soup = BeautifulSoup(all_problems.content, "html.parser")
    all_tags_div = soup.find(id = 'current-topic-tags')
    for tags_a_tag in all_tags_div.find_all('a'):
        tag = (tags_a_tag.span.text)
        url = ("https://leetcode.com/%s" % tags_a_tag['href'])
        yield  {'tag':tag, 'url':url}

抓取每一个Tag下的问题列表。

In [42]:
def crawle_tag(tag = None, url = None):
    html_content = requests.get(url)
    soup = BeautifulSoup(html_content.content, "html.parser")
    question_list = soup.find(id = 'question_list')
    question_list_body = question_list.tbody
    rows = question_list_body.find_all('tr')
    results = []
    for row in rows:
        question = {'tag':tag}
        cells = row.find_all('td')
        question['id'] = cells[1].text.strip()
        question['title'] = cells[2].a.text.strip()
        question['url'] = cells[2].a['href'].strip()
        question['ac'] = cells[3].text.strip()
        question['difficult'] = cells[4].text.strip()
        results.append(question)
    return results

执行前面两个函数，得到所有的问题。

In [47]:
questions = []
for taginfo in crawl_all_tags():
    tag = taginfo['tag']
    url = taginfo['url']
    results = crawle_tag(tag, url)
    questions.append(results)

# 数据分析

简单统计一下。

In [88]:
stat = {'ac':{'total':[], 'tag':{}, 'difficult':{}}, 'tag':{},'difficult':{}, 'count':0, "tag_diff":{}}

def p2f(p):
    return float(p.strip('%'))/100

for tagquestions in questions:
    for question in tagquestions:
        if question['difficult'] not in stat['difficult']:
            stat['difficult'][question['difficult']] = 0
        stat['difficult'][question['difficult']] += 1
        
        if question['tag'] not in stat['tag']:
            stat['tag'][question['tag']] = 0
        stat['tag'][question['tag']] += 1
        tag_diff = question['tag'] + "%%" + question['difficult']
        if tag_diff not in stat['tag_diff']:
            stat['tag_diff'][tag_diff] = 0
        stat['tag_diff'][tag_diff] += 1
        stat['count'] += 1
        
        ac = p2f(question['ac'])
        stat['ac']['total'].append(ac)
        if question['tag'] not in stat['ac']['tag']:
            stat['ac']['tag'][question['tag']] = []
        stat['ac']['tag'][question['tag']].append(ac)
        if question['difficult'] not in stat['ac']['difficult']:
            stat['ac']['difficult'][question['difficult']] = []
        stat['ac']['difficult'][question['difficult']].append(ac)

统计项如下：

1. 一共691个问题。其中容易的184个，难的143个，中等的364个，像是正态分布。
2. 最多的问题是数组相关的，104个。最少的是设计问题，32个。
3. 平均通过率0.37

In [89]:
print(stat['count'])
print(stat['difficult'])
print(stat['tag'])
print("%.2f" % np.mean(stat['ac']['total']))

691
{'Easy': 184, 'Hard': 143, 'Medium': 364}
{'Binary Search': 41, 'Depth-first Search': 57, 'Tree': 74, 'Math': 78, 'Dynamic Programming': 85, 'Backtracking': 35, 'Two Pointers': 37, 'Array': 104, 'Design': 32, 'Hash Table': 66, 'String': 82}
0.37


综合Tag和难度交叉来看排序。头等的还是数组类中等难度题。

In [90]:
tag_diff = sorted(stat['tag_diff'].items(), key = lambda x: x[1], reverse = True)

In [91]:
tag_diff

[('Array%%Medium', 57),
 ('Dynamic Programming%%Medium', 44),
 ('Tree%%Medium', 44),
 ('Math%%Medium', 40),
 ('Depth-first Search%%Medium', 37),
 ('String%%Medium', 36),
 ('Dynamic Programming%%Hard', 34),
 ('Array%%Easy', 33),
 ('Hash Table%%Medium', 29),
 ('Math%%Easy', 27),
 ('Hash Table%%Easy', 26),
 ('Tree%%Easy', 25),
 ('String%%Easy', 24),
 ('Backtracking%%Medium', 23),
 ('String%%Hard', 22),
 ('Binary Search%%Medium', 20),
 ('Two Pointers%%Medium', 18),
 ('Design%%Medium', 16),
 ('Array%%Hard', 14),
 ('Two Pointers%%Easy', 14),
 ('Backtracking%%Hard', 11),
 ('Depth-first Search%%Hard', 11),
 ('Hash Table%%Hard', 11),
 ('Math%%Hard', 11),
 ('Binary Search%%Easy', 11),
 ('Binary Search%%Hard', 10),
 ('Depth-first Search%%Easy', 9),
 ('Design%%Hard', 9),
 ('Design%%Easy', 7),
 ('Dynamic Programming%%Easy', 7),
 ('Tree%%Hard', 5),
 ('Two Pointers%%Hard', 5),
 ('Backtracking%%Easy', 1)]

看到这个，我想简单按照这样的思路来刷题：把容易度占比升序。

容易度定义可以有2个：

1. Tag的AC率均值
2. 1 - Tag的difficult题目比例

In [97]:
def sort_tag_by_ac_mean(stat):
    ac_mean = []
    for tag_ac in stat['ac']['tag']:
        ac = np.mean(stat['ac']['tag'][tag_ac])
        ac_mean.append((tag_ac, ac))
    return sorted(ac_mean, key = lambda x:x[1], reverse=True)

sort_tag_by_ac_mean(stat)

[('Tree', 0.41710810810810811),
 ('Hash Table', 0.3959545454545455),
 ('Depth-first Search', 0.39133333333333331),
 ('Array', 0.37540384615384614),
 ('Math', 0.3702948717948718),
 ('Two Pointers', 0.36091891891891892),
 ('Backtracking', 0.3597428571428572),
 ('Binary Search', 0.3554146341463415),
 ('String', 0.34948780487804876),
 ('Design', 0.34890625000000003),
 ('Dynamic Programming', 0.34762352941176472)]

看上去是是树相关的问题通过率最高，动态规划问题最难，和直觉相符。
再看我们用另一种定义来排序看看。

In [105]:
def sort_tag_by_easy_rate(stat):
    easy_rate = []
    tag_diff = {}
    for item in stat['tag_diff']:
        tag, diff = item.split('%%')
        if diff != 'Easy':
            continue
        if tag not in tag_diff:
            tag_diff[tag] = 0

        tag_diff[tag] += 1
    for tag in tag_diff:
        total = stat['tag'][tag]
        hard_count = tag_diff[tag]
        easy_rate.append((tag, float(hard_count + 5)/float(5 + total)))
    return sorted(easy_rate, key = lambda x: x[1], reverse=True)

sort_tag_by_easy_rate(stat)

[('Design', 0.16216216216216217),
 ('Backtracking', 0.15),
 ('Two Pointers', 0.14285714285714285),
 ('Binary Search', 0.13043478260869565),
 ('Depth-first Search', 0.0967741935483871),
 ('Hash Table', 0.08450704225352113),
 ('Tree', 0.0759493670886076),
 ('Math', 0.07228915662650602),
 ('String', 0.06896551724137931),
 ('Dynamic Programming', 0.06666666666666667),
 ('Array', 0.05504587155963303)]

至此我已经有点迷惑了，觉得这个题目的难度评级和大家实际表现出来的AC率完全不一致啊。

至此，我还是相信群众的AC率了，不相信专家的难度评级。按照以下顺序刷题：

In [106]:
sort_tag_by_ac_mean(stat)

[('Tree', 0.41710810810810811),
 ('Hash Table', 0.3959545454545455),
 ('Depth-first Search', 0.39133333333333331),
 ('Array', 0.37540384615384614),
 ('Math', 0.3702948717948718),
 ('Two Pointers', 0.36091891891891892),
 ('Backtracking', 0.3597428571428572),
 ('Binary Search', 0.3554146341463415),
 ('String', 0.34948780487804876),
 ('Design', 0.34890625000000003),
 ('Dynamic Programming', 0.34762352941176472)]